In [134]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn;
import sys
import nltk
nltk.download('stopwords')
from gensim.models import ldamodel
import gensim.corpora
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
import pickle

[nltk_data] Downloading package stopwords to /Users/mwang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [145]:
data = pd.read_csv('/Users/mwang/Documents/Mine/abcnews.csv', error_bad_lines=False)
data_text = data[['headline_text']]
print(data_text.head())

                                       headline_text
0  aba decides against community broadcasting lic...
1     act fire witnesses must be aware of defamation
2     a g calls for infrastructure protection summit
3           air nz staff in aust strike for pay rise
4      air nz strike to affect australian travellers


In [148]:
import random 
data_text = data_text.astype('str')
data_text2 = pd.DataFrame()
df = []
l = len(data_text)
for idx in range(10000):
    idxx = int(random.random()*l)
    df.append([word for word in data_text['headline_text'][idxx].split(' ') if word not in stopwords.words()])

In [149]:
data_text2['headline_text'] = df
#pickle.dump(data_text2, open('/Users/mwang/Documents/Mine/abcnews2.txt', 'wb'))
print(data_text2)

                                          headline_text
0              [citizenship, test, august, 2007, draft]
1                    [investigation, whales, stranding]
2     [bravery, awards, south, australians, honoured...
3                   [train, crashes, cleveland, staion]
4      [aust, extends, border, surveillance, contracts]
5     [swan, turnbull, trade, blows, likely, rate, r...
6                         [evans, wins, tour, romandie]
7     [x, ray, prompts, opening, tanner, shooting, c...
8                     [wilson, inlet, fish, get, clear]
9               [gaddafi, sold, 20, cent, libyas, gold]
10    [whale, rider, invited, join, hollywood, big, ...
11        [coonawarra, host, blue, gum, study, seminar]
12           [workers, consider, resuming, work, blaze]
13           [flooding, sparks, melbourne, rail, chaos]
14                                [farmers, keen, sell]
15           [protesters, rally, foreign, intervention]
16              [better, late, never, says, fisi

In [150]:
print(data_text2.iloc[0:1].values)
train_headlines = [value[0] for value in data_text2.iloc[0:].values]
print(train_headlines)

[[list(['citizenship', 'test', 'august', '2007', 'draft'])]]
[['citizenship', 'test', 'august', '2007', 'draft'], ['investigation', 'whales', 'stranding'], ['bravery', 'awards', 'south', 'australians', 'honoured', 'dealing', 'shar'], ['train', 'crashes', 'cleveland', 'staion'], ['aust', 'extends', 'border', 'surveillance', 'contracts'], ['swan', 'turnbull', 'trade', 'blows', 'likely', 'rate', 'rise'], ['evans', 'wins', 'tour', 'romandie'], ['x', 'ray', 'prompts', 'opening', 'tanner', 'shooting', 'case'], ['wilson', 'inlet', 'fish', 'get', 'clear'], ['gaddafi', 'sold', '20', 'cent', 'libyas', 'gold'], ['whale', 'rider', 'invited', 'join', 'hollywood', 'big', 'guns'], ['coonawarra', 'host', 'blue', 'gum', 'study', 'seminar'], ['workers', 'consider', 'resuming', 'work', 'blaze'], ['flooding', 'sparks', 'melbourne', 'rail', 'chaos'], ['farmers', 'keen', 'sell'], ['protesters', 'rally', 'foreign', 'intervention'], ['better', 'late', 'never', 'says', 'fisichella'], ['beaconsfield', 'hosts', 

In [131]:
num_topics = 10;
id2word = gensim.corpora.Dictionary(train_headlines);
corpus = [id2word.doc2bow(text) for text in train_headlines];
lda = ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics);

In [132]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

In [133]:
get_lda_topics(lda, num_topics)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,us,anti,sars,new,iraqi,baghdad,two,court,world,police
1,iraq,us,suicide,council,us,missing,death,woman,iraq,claims
2,iraqi,govt,suspected,qld,killed,police,police,faces,bush,inquiry
3,says,nsw,three,govt,community,centre,iraqi,help,saddam,assault
4,troops,plans,hospital,security,rally,title,toll,murder,cup,probe
5,forces,denies,cases,plan,season,may,virus,tv,back,final
6,attack,protest,australian,british,attacks,lead,us,car,tour,coast
7,still,fire,water,begins,govt,found,anti,trial,iraqis,victory
8,korea,says,shows,police,hopes,airport,killed,kuwait,support,nats
9,fire,port,food,dead,world,reports,protesters,held,south,aid


# NMF

In [140]:
train_headlines_sentences = [' '.join(text) for text in train_headlines]
vectorizer = CountVectorizer(analyzer='word', max_features=5000)
x_counts = vectorizer.fit_transform(train_headlines_sentences)
transformer = TfidfTransformer(smooth_idf=False)
x_tfidf = transformer.fit_transform(x_counts)
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

In [141]:
#obtain a NMF model.
model = NMF(n_components=num_topics, init='nndsvd');
#fit the model
model.fit(xtfidf_norm)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0,
  max_iter=200, n_components=10, random_state=None, shuffle=False,
  solver='cd', tol=0.0001, verbose=0)

In [142]:
def get_nmf_topics(model, n_top_words):
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    word_dict = {}
    for i in range(num_topics):
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words
    return pd.DataFrame(word_dict)

In [143]:
get_nmf_topics(model, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,us,police,iraq,govt,council,new,crash,test,water,charged
1,baghdad,probe,says,nsw,security,resolution,woman,real,rain,murder
2,iraqi,death,howard,vic,welcomes,ceo,car,first,plan,stabbing
3,troops,search,bush,urged,funding,world,hospital,waugh,restrictions,court
4,turkey,missing,pm,fire,decision,zealand,killed,australia,farmers,attack
5,forces,investigate,missiles,qld,elections,takes,two,future,drought,charge
6,korea,stabbing,post,nt,land,cup,accident,call,plans,assault
7,attack,anti,set,wa,call,nats,fatal,dope,may,attempted
8,military,shooting,blair,rejects,claims,launched,plane,clash,qld,robbery
9,killed,charge,aid,back,seeks,high,head,positive,hope,pair
